In [24]:
import xgboost as xgb
import numpy as np
import pandas as pd

In [25]:
raw_data = pd.read_csv('data/processed_data_ohe.csv')


In [50]:
data = raw_data[['loan_amnt', 'term', 'dti', 'int_rate', 'mo_sin_old_rev_tl_op', 'acc_open_past_24mths', 'total_bc_limit', 'home_ownership_RENT', 'annual_inc', 'delinq_2yrs', 'avg_cur_bal', 'num_accts_ever_120_pd', 'all_util', 'num_rev_tl_bal_gt_0', 'num_tl_120dpd_2m', 'purpose_moving', 'open_act_il', 'num_tl_op_past_12m', 'inq_last_6mths', 'num_bc_sats', 'loan_status']]
#data = raw_data
#data.columns

In [51]:
data.describe()

,loan_amnt,term,dti,int_rate,mo_sin_old_rev_tl_op,acc_open_past_24mths,total_bc_limit,annual_inc,delinq_2yrs,avg_cur_bal,num_accts_ever_120_pd,all_util,num_rev_tl_bal_gt_0,num_tl_120dpd_2m,open_act_il,num_tl_op_past_12m,inq_last_6mths,num_bc_sats,loan_status
count,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06
mean,1.441997e+04,4.179029e+01,1.828264e+01,1.323971e+01,1.814754e+02,4.692894e+00,2.163393e+04,7.624757e+04,3.177917e-01,1.348857e+04,5.095651e-01,5.817612e+01,5.592638e+00,8.200680e-04,2.792540e+00,2.178893e+00,6.550761e-01,4.734222e+00,1.996499e-01
std,8.717098e+03,1.026838e+01,1.115886e+01,4.768782e+00,9.210334e+01,3.130409e+00,2.117250e+04,6.992485e+04,8.779831e-01,1.587540e+04,1.291879e+00,1.323401e+01,3.133551e+00,2.913726e-02,1.890087e+00,1.798149e+00,9.377687e-01,2.890483e+00,3.997373e-01
min,5.000000e+02,3.600000e+01,-1.000000e+00,5.310000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.000000e+03,3.600000e+01,1.179000e+01,9.750000e+00,1.200000e+02,2.000000e+00,8.000000e+03,4.578000e+04,0.000000e+00,3.238000e+03,0.000000e+00,5.817612e+01,3.000000e+00,0.000000e+00,2.792540e+00,1.000000e+00,0.000000e+00,3.000000e+00,0.000000e+00
50%,1.200000e+04,3.600000e+01,1.762000e+01,1.274000e+01,1.690000e+02,4.000000e+00,1.580000e+04,6.500000e+04,0.000000e+00,8.279000e+03,0.000000e+00,5.817612e+01,5.000000e+00,0.000000e+00,2.792540e+00,2.000000e+00,0.000000e+00,4.000000e+00,0.000000e+00
75%,2.000000e+04,3.600000e+01,2.405000e+01,1.599000e+01,2.260000e+02,6.000000e+00,2.740000e+04,9.000000e+04,0.000000e+00,1.793100e+04,5.095651e-01,5.817612e+01,7.000000e+00,0.000000e+00,2.792540e+00,3.000000e+00,1.000000e+00,6.000000e+00,0.000000e+00
max,4.000000e+04,6.000000e+01,9.990000e+02,3.099000e+01,8.520000e+02,6.400000e+01,1.105500e+06,1.099920e+07,3.900000e+01,9.580840e+05,5.100000e+01,1.980000e+02,4.500000e+01,6.000000e+00,5.700000e+01,3.200000e+01,8.000000e+00,6.300000e+01,1.000000e+00


In [27]:
data.head(20)

,acc_now_delinq,acc_open_past_24mths,all_util,annual_inc,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,collections_12_mths_ex_med,delinq_2yrs,...,earliest_credit_line_years_9725 days,earliest_credit_line_years_9755 days,earliest_credit_line_years_9786 days,earliest_credit_line_years_9817 days,earliest_credit_line_years_9847 days,earliest_credit_line_years_9878 days,earliest_credit_line_years_9908 days,earliest_credit_line_years_9939 days,earliest_credit_line_years_9968 days,earliest_credit_line_years_9999 days
0,0.0,4.0,34.0,55000.0,20701.0,1506.0,37.2,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
1,0.0,4.0,29.0,65000.0,9733.0,57830.0,27.1,0.0,0.0,1.0,...,False,False,False,False,False,False,False,False,False,False
2,0.0,6.0,65.0,63000.0,31617.0,2737.0,55.9,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
3,0.0,10.0,78.0,104433.0,27644.0,4567.0,77.5,0.0,0.0,1.0,...,False,False,False,False,False,False,False,False,False,False
4,0.0,0.0,76.0,34000.0,2560.0,844.0,91.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
5,0.0,6.0,74.0,180000.0,30030.0,0.0,102.9,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
6,0.0,4.0,55.0,85000.0,17700.0,13674.0,5.7,0.0,0.0,1.0,...,False,False,False,False,False,False,False,False,False,False
7,0.0,7.0,46.0,85000.0,1997.0,8182.0,50.1,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
8,0.0,5.0,49.0,42000.0,28528.0,9966.0,41.4,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False
9,0.0,4.0,67.0,64000.0,19159.0,7940.0,77.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,False


In [18]:
data.to_csv('data/LC_top20.csv', index=False)

In [28]:
# flip loan status 0 to 1 and 1 to 0
data['loan_status'] = 1 - data['loan_status']

In [32]:
data['loan_status'].value_counts()

loan_status
0    1076751
1     268599
Name: count, dtype: int64

In [33]:
# split df into train and test, 80% train, 20% test, keep distribution of target feature loan_status the same 
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, random_state=42, test_size=0.2, stratify=data['loan_status'])

x_train, y_train = train_data.drop('loan_status', axis=1), train_data['loan_status']
x_test, y_test = test_data.drop('loan_status', axis=1), test_data['loan_status']

In [37]:
# implement xgboost on train data and evaluate on validation data
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=300, max_depth=8, scale_pos_weight=4, n_jobs=-1)   
xgb.fit(x_train, y_train)

y_xgb = xgb.predict(x_test)
y_xgb_proba = xgb.predict_proba(x_test)[:, 1]

In [38]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
print('Accuracy:', accuracy_score(y_test, y_xgb))
print('Precision:', precision_score(y_test, y_xgb))
print('Recall:', recall_score(y_test, y_xgb))
print('F1 score: ', f1_score(y_test, y_xgb))
print('AUC: ', roc_auc_score(y_test, y_xgb_proba))

Accuracy: 0.6825881740810942
Precision: 0.33981436544528026
Recall: 0.6256329113924051
F1 score:  0.44041566201907956
AUC:  0.7239337897526426


In [40]:
from rfpimp import *
imp = importances(xgb, x_test, y_test) # permutation

In [41]:
imp.head(20)

,Importance
Feature,
loan_amnt,0.0250
term,0.0144
dti,0.0126
mo_sin_old_rev_tl_op,0.0126
revol_bal,0.0070
total_bc_limit,0.0066
mo_sin_rcnt_tl,0.0064
bc_util,0.0056
emp_length,0.0056


In [10]:
# save model as pkl in models folder
import pickle
import joblib
# save the model
joblib.dump(xgb, "models/xgb_top20.pkl")

['models/xgb_top20.pkl']

In [12]:
y_xgb_train = xgb.predict(x_train)
y_xgb_train_proba = xgb.predict_proba(x_train)[:, 1]

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
print('Accuracy:', accuracy_score(y_train, y_xgb_train))
print('Precision:', precision_score(y_train, y_xgb_train))
print('Recall:', recall_score(y_train, y_xgb_train))
print('F1 score: ', f1_score(y_train, y_xgb_train))
print('AUC: ', roc_auc_score(y_train, y_xgb_train_proba))

Accuracy: 0.8081437915783997
Precision: 0.8155000712983624
Recall: 0.982586507329339
F1 score:  0.89128007105802
AUC:  0.7406387871538482


In [15]:
# value counts of loan_status in train_data
train_data['loan_status'].value_counts()

loan_status
1    861401
0    214879
Name: count, dtype: int64

In [17]:
data.describe()

,loan_amnt,term,dti,int_rate,mo_sin_old_rev_tl_op,acc_open_past_24mths,total_bc_limit,annual_inc,delinq_2yrs,avg_cur_bal,num_accts_ever_120_pd,all_util,num_rev_tl_bal_gt_0,num_tl_120dpd_2m,open_act_il,num_tl_op_past_12m,inq_last_6mths,num_bc_sats,loan_status
count,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06,1.345350e+06
mean,1.441997e+04,4.179029e+01,1.828264e+01,1.323971e+01,1.814754e+02,4.692894e+00,2.163393e+04,7.624757e+04,3.177917e-01,1.348857e+04,5.095651e-01,5.817612e+01,5.592638e+00,8.200680e-04,2.792540e+00,2.178893e+00,6.550761e-01,4.734222e+00,8.003501e-01
std,8.717098e+03,1.026838e+01,1.115886e+01,4.768782e+00,9.210334e+01,3.130409e+00,2.117250e+04,6.992485e+04,8.779831e-01,1.587540e+04,1.291879e+00,1.323401e+01,3.133551e+00,2.913726e-02,1.890087e+00,1.798149e+00,9.377687e-01,2.890483e+00,3.997373e-01
min,5.000000e+02,3.600000e+01,-1.000000e+00,5.310000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.000000e+03,3.600000e+01,1.179000e+01,9.750000e+00,1.200000e+02,2.000000e+00,8.000000e+03,4.578000e+04,0.000000e+00,3.238000e+03,0.000000e+00,5.817612e+01,3.000000e+00,0.000000e+00,2.792540e+00,1.000000e+00,0.000000e+00,3.000000e+00,1.000000e+00
50%,1.200000e+04,3.600000e+01,1.762000e+01,1.274000e+01,1.690000e+02,4.000000e+00,1.580000e+04,6.500000e+04,0.000000e+00,8.279000e+03,0.000000e+00,5.817612e+01,5.000000e+00,0.000000e+00,2.792540e+00,2.000000e+00,0.000000e+00,4.000000e+00,1.000000e+00
75%,2.000000e+04,3.600000e+01,2.405000e+01,1.599000e+01,2.260000e+02,6.000000e+00,2.740000e+04,9.000000e+04,0.000000e+00,1.793100e+04,5.095651e-01,5.817612e+01,7.000000e+00,0.000000e+00,2.792540e+00,3.000000e+00,1.000000e+00,6.000000e+00,1.000000e+00
max,4.000000e+04,6.000000e+01,9.990000e+02,3.099000e+01,8.520000e+02,6.400000e+01,1.105500e+06,1.099920e+07,3.900000e+01,9.580840e+05,5.100000e+01,1.980000e+02,4.500000e+01,6.000000e+00,5.700000e+01,3.200000e+01,8.000000e+00,6.300000e+01,1.000000e+00
